## Exercise Solutions for Support Vector Machines

#### Q1. What is the fundamental idea behind Support Vector Machines?

- The fundamental idea behind Support Vector Machines is to fit the widest possible street between two classes for classification and conversely, fit as many training instances on the street for regression. Since usually datasets are non-linear, hyper-parameters can be set to allow margin violations.

#### Q2. What is a suport vector?

- Support vectors are basically any instances that are located on the street. The decision boundary (widest possible street separating the two classes) is entirely dependent on these support vectors. 

#### Q3. Why is it important to scale the inputs when using SVMs?

- 